### Описание задачи и входных данных

Описание задачи: в данном задании необходимо оценить, насколько смешными становятся заголовки новостей, в которых были сделаны небольшие правки. Каждый исправленый заголовок оценивается пятью судьями, каждый из которых выставляет оценку по следующей шкале:

0 = Не смешно

1 = Немного смешно

2 = Довольно смешно

3 = Очень смешно

Итоговая оценка измененного заголовка = среднее арифметическое оценок 5 судей


Соревнование имеет 2 подзадачи

Задача 1: Даны оригинальный и измененный заголовки, необходимо предсказать итоговую оценку (то есть среднее арифметическое оценок судей). Точность оценивается с помощью RMSE, также дополнительно оценивается RMSE N% самых смешных и самых несмешных замен (N = {10, 20, 30, 40}).

Задача 2: Даны две возможных замены заголовка, необходимо выбрать наиболее смешной из них. Система оценивается исходя из точности предсказания наиболее смешного заголовка (пары с одинаковыми оценками будут игнорироваться в оценке точности). Также дополнительно вычисляется дополнительная метрика "вознаграждение" (reward), где для правильного ответа вычисляется разница между оценками заголовков со знаком "+", а для неправильного - со знаком "-"




Входные данные:

Входные данные состоят примерно из 5000 оригинальных заголовков, для каждого заголовка имеется 3 возможных замены (то есть всего около 15000 заголовков). Данные разделены на обучающую, контрольную и тестовую выборки в отношении 64/16/20.

Для задачи 1 данные имеют следующие поля:


1.   id - уникальный номер заголовка с заменой
2.   original - оригинальный заголовок, в котором заменяемое слово заключено в тег </>
3.   edit - новое слово
4.   grades - оценки судей, записанные в 1 строчку, например 233332
5.   meanGrade - средняя оценка судей


Для задачи 2 данные имеют следующие поля:


1.   id - уникальный номер заголовка 1 - уникальный номер заголовка 2 
2.   label - может принимать несколько значений

0 - заголовки одинаково смешные

1 - замена 1 смешнее

2 - замена 2 смешнее

В этой работе рассматривается только 1 задание



In [0]:
from google.colab import files
uploaded = files.upload()

Saving dev.csv to dev (1).csv
Saving train.csv to train (1).csv


In [0]:
import pandas as pd

In [0]:
task1_train = pd.read_csv('train.csv')
task1_dev = pd.read_csv('dev.csv')

In [0]:
task1_train

,id,original,edit,grades,meanGrade
0,14530,France is ‘ hunting down its citizens who join...,twins,10000,0.2
1,13034,"Pentagon claims 2,000 % increase in Russian tr...",bowling,33110,1.6
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor ...,party,22100,1.0
3,76,"In an apparent first , Iran and Israel <engage...",slap,20000,0.4
4,6164,Trump was told weeks ago that Flynn misled <Vi...,school,0,0.0
...,...,...,...,...,...
9647,10899,State officials blast ' unprecedented ' DHS <m...,idea,0,0.0
9648,1781,Protesters Rally for <Refugees/> Detained at J...,stewardesses,20000,0.4
9649,5628,Cruise line Carnival Corp. joins the fight aga...,raisin,21000,0.6
9650,14483,Columbia police hunt woman seen with <gun/> ne...,cake,32200,1.4


In [0]:
task2_train = pd.read_csv('train (1).csv')
task2_dev = pd.read_csv('dev (1).csv')

In [0]:
task2_train

,id,original1,edit1,grades1,meanGrade1,original2,edit2,grades2,meanGrade2,label
0,10920-9866,""" Gene Cernan , Last <Astronaut/> on the Moon ...",Dancer,1113,1.2,""" Gene Cernan , Last Astronaut on the Moon , <...",impregnated,30001,0.8,1
1,3176-10722,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,0
2,3176-3702,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
3,10722-3702,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
4,12282-2083,""" Our expectations of what civic engagement lo...",imagine,0,0.0,""" Our expectations of what civic engagement <l...",smells,100220010,0.6,2
...,...,...,...,...,...,...,...,...,...,...
9376,975-13357,"“ It ’s painfully obvious "" Mueller will charg...",battery,1,0.2,"“ It ’s painfully obvious "" Mueller will charg...",plumbing,11103,1.2,2
9377,975-11773,"“ It ’s painfully obvious "" Mueller will charg...",battery,1,0.2,"“ It ’s painfully obvious "" Mueller will <char...",strangle,22331,2.2,2
9378,13357-11773,"“ It ’s painfully obvious "" Mueller will charg...",plumbing,11103,1.2,"“ It ’s painfully obvious "" Mueller will <char...",strangle,22331,2.2,2
9379,14954-14479,"“ Kompromat , ” media ethics and the law : Wha...",porn,20101,0.8,"“ Kompromat , ” media ethics and the law : Wha...",dance,32112,1.8,2


Средняя оценка замен в 1 подзадаче:

In [0]:
task1_train['meanGrade'].mean()

0.9355712114932938

Так как замены оцениваются субъективно, не представляется возможным оценить корректность и ошибки в данных.

In [0]:
print(len(task2_train[task2_train['label'] == 1]))
print(len(task2_train[task2_train['label'] == 2]))
print(len(task2_train[task2_train['label'] == 0]))

4198
4184
999


Во второй подзадаче классы 1 и 2 примерно имеют примерно одинаковое количество элементов.

###Обзор литературы

#####1.SARCASM AND HUMOR DETECTION USING MACHINE LEARNING"

В статье "SARCASM AND HUMOR DETECTION USING MACHINE LEARNING" оценивается чувство юмора в предложениях со структурой (NN,V,JJ,NN). Использовались  StanfordCoreNLP и методы word2vec, LSA, ESA и GLOVE.

![Архитектура модели](https://drive.google.com/uc?export=view&id=1Ambwea83WeWS8uidCCUDtM3GaTYF4Pql)

Для классификации юмора использовался наивный байесовский классификатор. После обучения на униграммах получали точность 76.6%, на биграммах и триграммах - 35%.

##### 2. Automatic Narrative Humor Recognition Method Using Machine Learning and Semantic Similarity Based Punchline Detection

В работе "Automatic Narrative Humor Recognition Method Using Machine Learning and
Semantic Similarity Based Punchline Detection" оцениваются тексты (короткие истории) на предмет юмора. Выборка состоит из 987 текстов, найденных по запросу "funny stories" (смешные тексты) и 1211 твитов, найденых по хэштегу #twnovel (несмешные тексты). Далее полученные тексты оценивались 11 людьми, и если больше половины соглашались с каким-либо вердиктом, это становилось меткой текста. В работе использовались SVM и наивный байесовский классификатор. Если их вердикты различались, то вводился дополнительный модуль Punchline Detection - если первая половина текста сильно отличалась от второй, то считалось, что в тексте есть панчлайн, и в тексте есть юмор. 
 

 ![модель](https://drive.google.com/uc?export=view&id=138r8ttK1cZ7n5AfLpuR2u8nFJqDjpbLU)

#### Базовая архитектура модели

Аналогично рассмотренным моделям, предлагается использовать наивный байесовский классификатор. Для первой подзадачи сначала просто раздавать метки 3 и 0 (в дальнейшем эту часть необходимо модифицировать для выдачи потенциальной средней оценки экспертов).

Для второй подзадачи также используется наивный байесовский классификатор, выдающий метки 0, 1 или 2.

##### Baseline

Baseline в первой подзадаче: всегда предсказывает среднее всех оценок обучающей выборки Score = 0.5783998503

Baseline второй подзадачи: предсказывает самую популярную категорию второй выборки Score = 0.5140543116	

# LSTM

В этой части используется двухсторонняя LSTM с 32 слоями и одним слоем Dropout. Все заголовки были конвертированны в векторы с помощью GloVe. Оценивался уже обработанный заголовок (то есть заголовок, в котором вместо старого слова поставленно новое).
LSTM лучше подходит для анализа текста, так как данная модель создана для обработки последовательных данных (в т.ч. текстов)

In [0]:
def bins(score):
    if (score <= 0.15):
        return 0
    elif (score <= 0.3):
        return 1
    elif (score <= 0.45):
        return 2
    elif (score <= 0.6):
        return 3
    elif (score <= 0.75):
        return 4
    elif (score <= 0.9):
        return 5
    elif (score <= 1.05):
        return 6
    elif (score <= 1.2):
        return 7
    elif (score <= 1.35):
        return 8
    elif (score <= 1.5):
        return 9
    elif (score <= 1.65):
        return 10
    elif (score <= 1.8):
        return 11
    elif (score <= 1.95):
        return 12
    elif (score <= 2.1):
        return 13
    elif (score <= 2.25):
        return 14
    elif (score <= 2.4):
        return 15
    elif (score <= 2.55):
        return 16
    elif (score <= 2.7):
        return 17
    elif (score <= 2.85):
        return 18
    else:
        return 19

max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32



In [0]:
def debins(binned):
    if (binned == 0):
        return 0
    elif(binned==1):
        return 0.225
    elif(binned==2):
        return 0.375
    elif(binned==3):
        return 0.525
    elif(binned==4):
        return 0.675
    elif(binned==5):
        return 0.825
    elif(binned==6):
        return 0.975
    elif(binned==7):
        return 1.125
    elif(binned==8):
        return 1.275
    elif(binned==9):
        return 1.425
    elif(binned==10):
        return 1.575
    elif(binned==11):
        return 1.725
    elif(binned==12):
        return 1.875
    elif(binned==13):
        return 2.025
    elif(binned==14):
        return 2.175
    elif(binned==15):
        return 2.325
    elif(binned==16):
        return 2.475
    elif(binned==17):
        return 2.625
    elif(binned==18):
        return 2.775
    elif(binned==19):
        return 3
    

In [0]:
edited_lines = []
for i, rows in task1_train.iterrows():
    line = rows['original'].split()
    edited_line = []
    for word in line:
        if '<' in word:
            edited_line.append(rows['edit'])
        else:
            edited_line.append(word)
    edited_lines.append(edited_line)

In [0]:
dev_lines = []
for i, rows in task1_dev.iterrows():
    line = rows['original'].split()
    edited_line = []
    for word in line:
        if '<' in word:
            edited_line.append(rows['edit'])
        else:
            edited_line.append(word)
    dev_lines.append(edited_line)

In [111]:
edited_lines[1]

['Pentagon',
 'claims',
 '2,000',
 '%',
 'increase',
 'in',
 'Russian',
 'trolls',
 'after',
 'bowling',
 'strikes',
 '.',
 'What',
 'does',
 'that',
 'mean',
 '?']

In [0]:
lemmatized_corpus = []
for sentence in edited_lines:
    new_sentence = []
    for word in sentence:
        new_sentence.append(lemmatizer.lemmatize(word))
    lemmatized_corpus.append(new_sentence)

In [0]:
lemmatized_dev = []
for sentence in dev_lines:
    new_sentence = []
    for word in sentence:
        new_sentence.append(lemmatizer.lemmatize(word))
    lemmatized_dev.append(new_sentence)

In [0]:
for sentence in lemmatized_corpus:
    for i in range(len(sentence)):
        sentence[i] = sentence[i].lower()
        sentence[i] = re.sub("[^a-zA-Z]+", "",  sentence[i])


In [0]:
for sentence in lemmatized_dev:
    for i in range(len(sentence)):
        sentence[i] = sentence[i].lower()
        sentence[i] = re.sub("[^a-zA-Z]+", "",  sentence[i])


In [0]:
binned_score = []
for i, rows in task1_train.iterrows():
    binned_score.append(bins(rows['meanGrade']))

In [117]:
binned_score[:5]

[1, 10, 6, 2, 0]

In [118]:
len(lemmatized_corpus)

9652

In [0]:
from glove import Corpus, Glove

In [120]:
corpus = Corpus() 

#Training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lemmatized_corpus, window=10)

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')


Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [0]:
files.download('glove.model')

In [0]:
import inspect
inspect.getmembers(glove)

In [0]:
f = open('glovey.txt', 'w')
for i in inspect.getmembers(glove):
    f.write(str(i))
    f.write('\n')
#f.write(str(inspect.getmembers(glove)))
f.close
files.download('glovey.txt')

In [0]:
import glove as gl

In [125]:
glove.word_vectors[0]

array([-0.3374837 , -0.07727461, -0.15514989,  0.17236857,  0.24468199])

In [126]:
glove.word_vectors[glove.dictionary['man']]

array([-0.19124603, -0.11154896, -0.23371708,  0.17132904,  0.24269754])

In [0]:
tru_matrix = []
for i in range(len(lemmatized_corpus)):
    sentence_matrix = []
    for word in lemmatized_corpus[i]:
        sentence_matrix.append(glove.word_vectors[glove.dictionary[word]])
    tru_matrix.append(sentence_matrix)

In [0]:
dev_matrix = []
for i in range(len(lemmatized_dev)):
    sentence_matrix = []
    for word in lemmatized_dev[i]:
        try:
            sentence_matrix.append(glove.word_vectors[glove.dictionary[word]])
        except:
            sentence_matrix.append(Xpad)
    dev_matrix.append(sentence_matrix)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(tru_matrix, binned_score, test_size=0.2, random_state=42)

In [131]:
X_train[0]

[array([-0.3265755 , -0.35068634, -0.24532761,  0.12896453, -0.36516801]),
 array([-0.55671406, -0.60688291, -0.34049844,  0.46407153,  0.01641401]),
 array([-1.23345774, -1.20842644, -0.3100262 ,  0.96252704,  0.62057601]),
 array([-0.12617974, -0.07316746, -0.03391419,  0.05279105,  0.02319443]),
 array([-0.47082537, -0.25951529, -0.18568189,  0.31949243,  0.24521519]),
 array([-0.91476332, -0.79555142, -0.33606053,  0.81129011,  0.66139212]),
 array([-0.9240497 , -0.98286301, -0.2923284 ,  0.73529676,  0.81576535]),
 array([0.00378085, 0.01886009, 0.038614  , 0.04513735, 0.12923635]),
 array([-0.09243699, -0.18686961, -0.08175656,  0.08549737,  0.21033011]),
 array([-1.17358357, -1.03533567, -1.83905585,  0.99582145,  1.62208364]),
 array([-0.03782027, -0.06240072, -0.35813526,  0.1207342 ,  0.20202424]),
 array([-0.01794078, -0.08144791, -0.02334543,  0.08904117,  0.01926065]),
 array([-0.62960605, -1.15860405, -0.75250542,  1.0605674 ,  0.97966534]),
 array([-1.17358357, -1.035335

In [0]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32
N=1000

In [0]:
max_seq_len2 = max(map(len, X_test))

In [134]:
max_seq_len2

26

In [0]:
special_value = -10.0
max_seq_len = max(map(len,X_train)) 
Xpad = [-10] * 5

In [0]:
for i in range(0, len(X_train)):
    if (len(X_train[i]) < 26):
        for j in range(len(X_train[i]), 26):
            #print(X_train[i])
            X_train[i].append(Xpad)

In [0]:
for i in range(0, len(X_test)):
    if (len(X_test[i]) < 26):
        for j in range(len(X_test[i]), 26):
            X_test[i].append(Xpad)

In [0]:
for i in range(0, len(dev_matrix)):
    if (len(dev_matrix[i]) < 26):
        for j in range(len(dev_matrix[i]), 26):
            dev_matrix[i].append(Xpad)

In [0]:
np_train = np.array(X_train)

In [0]:
np_test = np.array(X_test)

In [0]:
np_dev = np.array(dev_matrix)

In [142]:
np_train[0].shape

(26, 5)

In [0]:
model = Sequential()
#model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Masking(mask_value=special_value, input_shape=(26, 5)))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])


In [144]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_5 (Masking)          (None, 26, 5)             0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                9728      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 9,793
Trainable params: 9,793
Non-trainable params: 0
_________________________________________________________________


In [145]:
model.fit(np_train, y_train,
          batch_size=batch_size,
          epochs=25,
          validation_data=[np_test, y_test])

Train on 7721 samples, validate on 1931 samples
Epoch 1/25
7721/7721 [==============================] - 29s 4ms/step - loss: -63.5570 - acc: 0.0881 - val_loss: -73.8623 - val_acc: 0.0922
Epoch 2/25
7721/7721 [==============================] - 27s 3ms/step - loss: -73.2851 - acc: 0.0993 - val_loss: -74.1820 - val_acc: 0.0922
Epoch 3/25
7721/7721 [==============================] - 27s 3ms/step - loss: -73.4806 - acc: 0.0859 - val_loss: -74.3681 - val_acc: 0.0528
Epoch 4/25
7721/7721 [==============================] - 27s 3ms/step - loss: -73.5527 - acc: 0.0745 - val_loss: -74.3736 - val_acc: 0.0445
Epoch 5/25
7721/7721 [==============================] - 27s 4ms/step - loss: -73.5545 - acc: 0.0760 - val_loss: -74.3883 - val_acc: 0.0352
Epoch 6/25
7721/7721 [==============================] - 27s 3ms/step - loss: -73.5714 - acc: 0.0787 - val_loss: -74.3889 - val_acc: 0.0378
Epoch 7/25
7721/7721 [==============================] - 27s 4ms/step - loss: -73.5688 - acc: 0.0730 - val_loss: -74.38

In [0]:
model.save('lstm.h5')

In [0]:
files.download('lstm.h5')

In [149]:
score = model.evaluate(np_test, y_test, batch_size=128)


1931/1931 [==============================] - 1s 296us/step


In [150]:
score

[-74.40330469812031, 0.05592957018632953]

In [0]:
pred = model.predict(np_dev)

In [152]:
pred

array([[10.024673 ],
       [ 7.4790225],
       [10.371142 ],
       ...,
       [10.6318245],
       [10.717829 ],
       [ 9.546591 ]], dtype=float32)

In [153]:
pred.mean()

9.991202

In [0]:
output_table = pd.DataFrame()
output_table['id'] = task1_dev['id']
labels = []
for i in range(len(task1_dev)):
    labels.append(debins(20 - round(pred[i][0])))

In [0]:
output_table['pred'] = labels

In [166]:
output_table

,id,pred
0,1723,1.575
1,12736,2.025
2,12274,1.575
3,8823,1.425
4,5087,1.425
...,...,...
2414,1202,1.575
2415,14764,1.425
2416,12595,1.425
2417,70,1.425


In [0]:
output_table.to_csv('task-1-output.csv')
files.download('task-1-output.csv')

скриншот посылок в Codalab
https://imgur.com/YSTZqrO

скриншот лидерборда https://imgur.com/5CN1Fr0

#### Итоги

В результате работы разработана модель обработки текста и оценки юмора микрозамен коротких предложений (до 26 слов). В дальнейшем модель можно улучшить, чтобы добиться лучших результатов.